In [1]:
%reset
!rm -rf '/content/gaf_output'
!rm '/content/gaf_output.zip'
%cd /content/

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
rm: cannot remove '/content/gaf_output.zip': No such file or directory
/content


In [9]:
# Constants
## Folder names
folder_name = 'gaf_output'
training_folder_name = 'training'
test_folder_name = 'test'

## Imported dataset
parameter_name = 'state'
col_name = ['datetime','state','ts']
convert_dict = {'ts': int,
                'state': int}

# Dataset files:
# home_assistant_v2_01_09_2022_localbytes_power.csv: 19uBe50RIHwHoIuP12O_EHlQTmcYuDBHD
# home_assistant_v2_01_09_2022_localbytes2_power.csv: 1YKZzFK0fS0OjpFMKODYjcG8rJgiz1aeK
# home_assistant_v2_01_09_2022_localbytes3_power.csv: 1j9mJn2lkFcMS7XuzSOrLHni6JtMKbTe8
# home_assistant_v2_01_09_2022_localbytes4_power.csv: 1SRvrmBcgTtko8H4bllama-Xi43dPSM2X
# home_assistant_v2_01_09_2022_localbytes5_power.csv: 1G3OtoOgq1F-QvWTqYvdoxtyRn2iKplSg
# home_assistant_v2_01_09_2022_localbytes7_power.csv: 1DUcaPi7ZiO1SgMet3YhnDM_nozAMtnpP
# home_assistant_v2_01_09_2022_localbytes8_power.csv: 1TQY3DyFsn_p4BmhVsLjbEFNfEAgiHYtC
# home_assistant_v2_01_09_2022_ewelink_th01_8dc96c24_humidity: 1J15Rnt3gRSyskCf_gcKQHHcwDwffL-tp
# home_assistant_v2_01_09_2022_ewelink_th01_8dc96c24_temperature: 1QtN45ZpAvrWK19FEv2Tm7kgR2F5hRU-Q

file_id = '1j9mJn2lkFcMS7XuzSOrLHni6JtMKbTe8'
filename = '/content/drive/MyDrive/Colab Notebooks/data/home_assistant_v2_01_09_2022_localbytes3_power.csv'

## GAF parameters
training_output_dataset_filename = 'gaf_dataset_training.csv'
test_output_dataset_filename = 'gaf_dataset_test.csv'
data_batch = 100000 #70000000
GAF_RESOLUTION = 128
IMAGE_SIZE= (5,5) # in inches

NORMAL_CLASS_LABEL = 'normal'
NORMAL_CLASS_LABEL_NO = 0
ABNORMAL_CLASS_LABEL = 'abnormal'
ABNORMAL_CLASS_LABEL_NO = 1
frequency = '1h'

eval_split = 0.1
training_split = 0.9

In [3]:
# Install required libaries
!pip install -q pyts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.2 MB/s eta 0:00:00


In [4]:
# Imports
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

import numpy as np
import pandas as pd
from datetime import datetime
import os
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from pyts.image import GramianAngularField
from typing import *
from multiprocessing import Pool
import datetime as dt
matplotlib.use('Agg')

In [5]:
# Import data
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
dataset_downloaded = drive.CreateFile({'id': file_id})
dataset_downloaded.GetContentFile(filename)

In [10]:
# Preprocessing
df = pd.read_csv(filename, names=col_name, header=None)
print(df.head())
convert_dict = {'ts': int,
                'state': int}
df = df.head(data_batch)
df = df.astype(convert_dict)
df['datetime'] = df['ts'].apply(lambda x: datetime.utcfromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))
print(df.size, df.head())

                     datetime  state          ts
0  2022-04-22 13:06:14.576056      0  1650632774
1  2022-04-22 13:10:35.108171      0  1650633035
2  2022-04-22 14:00:02.244148      0  1650636002
3  2022-04-22 14:28:29.815314      0  1650637709
4  2022-04-22 14:29:20.267582      0  1650637760
300000               datetime  state          ts
0  2022-04-22 13:06:14      0  1650632774
1  2022-04-22 13:10:35      0  1650633035
2  2022-04-22 14:00:02      0  1650636002
3  2022-04-22 14:28:29      0  1650637709
4  2022-04-22 14:29:20      0  1650637760


In [11]:
# Folder setup
print('Creating Directories...')

PATH = os.path.abspath('')
GAF_PATH = os.path.join(PATH , folder_name)

TRAINING_OUTPUT_PATH = os.path.join(GAF_PATH , training_folder_name)
TEST_OUTPUT_PATH = os.path.join(GAF_PATH , test_folder_name)

TRAINING_NORMAL_PATH = os.path.join(TRAINING_OUTPUT_PATH , NORMAL_CLASS_LABEL)
TRAINING_ABNORMAL_PATH = os.path.join(TRAINING_OUTPUT_PATH , ABNORMAL_CLASS_LABEL)

os.makedirs(TRAINING_OUTPUT_PATH, exist_ok=True)
os.makedirs(TEST_OUTPUT_PATH, exist_ok=True)
os.makedirs(TRAINING_NORMAL_PATH, exist_ok=True)
os.makedirs(TRAINING_ABNORMAL_PATH, exist_ok=True)

Creating Directories...


In [12]:
# Create and save GAFs - supporting functions
def create_gaf(ts) -> Dict[str, Any]:
    """
    :param ts:
    :return:
    """
    data = dict()
    gasf = GramianAngularField(method='summation', image_size=ts.shape[0])
    data['gasf'] = gasf.fit_transform(pd.DataFrame(ts).T)[0]
    data['gasf_mean'] = np.mean(np.mean(data['gasf'], axis=0), axis=0)
    return data


# Create images of the bundle that we pass
def create_images(X_plots: Any, image_name: str, output_path: str, destination: str, image_matrix: tuple =(1, 1), mix_classes: bool = False) -> None:
    """
    :param X_plots:
    :param image_name:
    :param destination:
    :param image_matrix:
    :return:
    """
    # output_dataset_image_list.append(image_name)
    fig = plt.figure(figsize=IMAGE_SIZE, frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    images = X_plots
    ax.imshow(images[0], cmap='rainbow', origin='lower')

    repo = os.path.join(output_path, destination) if mix_classes == False else output_path
    fig.savefig(os.path.join(repo, image_name), bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    return {'image_name': image_name+'.png', 'label': destination}

def generate_gaf(images_data: Dict[str, pd.DataFrame]) -> None:
    """
    :param images_data:
    :return:
    """
    print('Starting image generation...')
    output_dataset = pd.DataFrame(columns = ['image_name','label'])
    main_decision = list(images_data.keys())[0]
    for decision, data in images_data.items():
        for i, image_data in enumerate(data):
            # print('decision', decision)
            # print('image_data', image_data)
            to_plot = [create_gaf(x)['gasf'] for x in image_data[1]]
            gaf_mean = [create_gaf(x)['gasf_mean'] for x in image_data[1]]
            classfication = NORMAL_CLASS_LABEL if gaf_mean[0] < 0 else ABNORMAL_CLASS_LABEL

            print(i, '- gaf_mean:', gaf_mean, '-> classification:', classfication)
            i = len(output_dataset)
            output_path = TRAINING_OUTPUT_PATH if decision == training_folder_name else TEST_OUTPUT_PATH
            destination_folder = classfication
            mix_classes = False if decision == training_folder_name else True
            output_dataset.loc[i] = create_images(X_plots=to_plot,
                              image_name='{0}'.format(image_data[0].replace('-', '_')),
                              output_path=output_path, destination=destination_folder, mix_classes=mix_classes)

    def label_no (row):
      if row['label'] == NORMAL_CLASS_LABEL :
          return NORMAL_CLASS_LABEL_NO
      elif row['label'] == ABNORMAL_CLASS_LABEL:
        return ABNORMAL_CLASS_LABEL_NO
      else:
        return ABNORMAL_CLASS_LABEL_NO

    output_dataset['label_no'] = output_dataset.apply (lambda row: label_no(row), axis=1)

    # Convert output_dataset to csv
    dataset_filename = GAF_PATH + '/' + training_output_dataset_filename if main_decision == training_folder_name else GAF_PATH + '/' + test_output_dataset_filename
    output_dataset.to_csv(dataset_filename, index=False)

In [13]:
# Create and save GAFs
def data_to_image_preprocess(data: pd.DataFrame, create_eval: bool = False, destination_folder: str = 'training') -> None:
    """
    :return: None
    """
    print('Processing dataset...')
    # Drop unnecessary data_slice
    data['DateTime'] = pd.to_datetime(data['datetime'], infer_datetime_format=True)
    data = data.groupby(pd.Grouper(key='DateTime', freq=frequency)).mean().interpolate().reset_index()
    # print(df)
    # Send to slicing
    set_gaf_data(data, create_eval, destination_folder)


def set_gaf_data(df: pd.DataFrame, create_eval: bool = False, destination_folder: str = 'training') -> None:
    """
    :param df: DataFrame data_slice
    :return: None
    """
    dates = df['DateTime']
    print(dates)
    # dates = dates.drop_duplicates()
    list_dates = dates.apply(str).tolist()
    print('original list_dates:', len(list_dates))
    eval_split_len = round(len(list_dates) * eval_split) + 1
    training_split_len = round(len(list_dates) * training_split) + 1
    list_dates = list_dates[:training_split_len] if create_eval == False else list_dates[:eval_split_len]
    print('final list_dates:', len(list_dates))
    index = 0
    box_size = GAF_RESOLUTION
    # Container to store data_slice for the creation of GAF
    decision_map = {key: [] for key in [destination_folder]}
    print('list_dates:', list_dates)
    while True:
        if index >= len(list_dates) - 2:
            break
        # Select appropriate timeframe
        data_slice = df.loc[(df['DateTime'] < list_dates[len(list_dates) - 1]) & (df['DateTime'] > list_dates[index])]
        # print("DATA SLICE==========================================", index)
        # print(data_slice)
        # print("DATA SLICE END======================================")
        gafs = []
        # Group data_slice by time frequency
        for freq in [frequency]:
            group_dt = data_slice.groupby(pd.Grouper(key='DateTime', freq=freq)).mean().reset_index()
            group_dt = group_dt.dropna()
            gafs.append(group_dt[parameter_name].head(box_size))
            # print('group_dt', group_dt[parameter_name].head(box_size))
        decision_map[destination_folder].append([list_dates[index], gafs])
        index += 1

    # print('decision_map:', decision_map)
    print('Generating GAF images...')
    # Generate the images from processed data_slice
    generate_gaf(decision_map)
    total_images = len(decision_map[destination_folder])
    images_created = total_images
    print("========GAF REPORT========:\nTotal Images Created: {0}".format(images_created))

def main():
    global df
    print('CONVERTING TIME-SERIES TO GAF IMAGES...')
    print('Training dataset:')
    data_to_image_preprocess(data = df, create_eval = False, destination_folder = training_folder_name)
    print("-----------------------------------------------Training dataset created----------------------------------")
    print('Test dataset:')
    data_to_image_preprocess(data = df, create_eval = True, destination_folder = test_folder_name)
    print("-----------------------------------------------Test dataset created--------------------------------------")

main()

CONVERTING TIME-SERIES TO GAF IMAGES...
Training dataset:
Processing dataset...
0      2022-04-22 13:00:00
1      2022-04-22 14:00:00
2      2022-04-22 15:00:00
3      2022-04-22 16:00:00
4      2022-04-22 17:00:00
               ...        
1920   2022-07-11 13:00:00
1921   2022-07-11 14:00:00
1922   2022-07-11 15:00:00
1923   2022-07-11 16:00:00
1924   2022-07-11 17:00:00
Name: DateTime, Length: 1925, dtype: datetime64[ns]
original list_dates: 1925
final list_dates: 1733
list_dates: ['2022-04-22 13:00:00', '2022-04-22 14:00:00', '2022-04-22 15:00:00', '2022-04-22 16:00:00', '2022-04-22 17:00:00', '2022-04-22 18:00:00', '2022-04-22 19:00:00', '2022-04-22 20:00:00', '2022-04-22 21:00:00', '2022-04-22 22:00:00', '2022-04-22 23:00:00', '2022-04-23 00:00:00', '2022-04-23 01:00:00', '2022-04-23 02:00:00', '2022-04-23 03:00:00', '2022-04-23 04:00:00', '2022-04-23 05:00:00', '2022-04-23 06:00:00', '2022-04-23 07:00:00', '2022-04-23 08:00:00', '2022-04-23 09:00:00', '2022-04-23 10:00:00', '20

<ipython-input-13-0d967f448975>:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = data.groupby(pd.Grouper(key='DateTime', freq=frequency)).mean().interpolate().reset_index()


Generating GAF images...
Starting image generation...
0 - gaf_mean: [-0.008874054562500547] -> classification: normal
1 - gaf_mean: [0.05590086414011761] -> classification: abnormal
2 - gaf_mean: [0.04745703704579417] -> classification: abnormal
3 - gaf_mean: [0.05584339806776813] -> classification: abnormal
4 - gaf_mean: [-0.6642152044818044] -> classification: normal
5 - gaf_mean: [-0.6550756152963797] -> classification: normal
6 - gaf_mean: [-0.6440455963716054] -> classification: normal
7 - gaf_mean: [-0.6360693432246564] -> classification: normal
8 - gaf_mean: [-0.6666501091629953] -> classification: normal
9 - gaf_mean: [-0.6898187503938827] -> classification: normal
10 - gaf_mean: [-0.707792559505418] -> classification: normal
11 - gaf_mean: [-0.7216246719380183] -> classification: normal
12 - gaf_mean: [-0.732054280883038] -> classification: normal
13 - gaf_mean: [-0.7302111329737601] -> classification: normal
14 - gaf_mean: [-0.730445534320042] -> classification: normal
15 - g

<ipython-input-13-0d967f448975>:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = data.groupby(pd.Grouper(key='DateTime', freq=frequency)).mean().interpolate().reset_index()


Generating GAF images...
Starting image generation...
0 - gaf_mean: [-0.008874054562500547] -> classification: normal
1 - gaf_mean: [0.05590086414011761] -> classification: abnormal
2 - gaf_mean: [0.04745703704579417] -> classification: abnormal
3 - gaf_mean: [0.05584339806776813] -> classification: abnormal
4 - gaf_mean: [-0.6642152044818044] -> classification: normal
5 - gaf_mean: [-0.6550756152963797] -> classification: normal
6 - gaf_mean: [-0.6440455963716054] -> classification: normal
7 - gaf_mean: [-0.6360693432246564] -> classification: normal
8 - gaf_mean: [-0.6666501091629953] -> classification: normal
9 - gaf_mean: [-0.6898187503938827] -> classification: normal
10 - gaf_mean: [-0.707792559505418] -> classification: normal
11 - gaf_mean: [-0.7216246719380183] -> classification: normal
12 - gaf_mean: [-0.732054280883038] -> classification: normal
13 - gaf_mean: [-0.7302111329737601] -> classification: normal
14 - gaf_mean: [-0.730445534320042] -> classification: normal
15 - g

In [ ]:
# Save output images as a compressed file and dataset as CSV
!zip -r -q /content/gaf_output.zip /content/gaf_output
files.download("/content/gaf_output.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive', force_remount=False)
# %cd /gdrive/My\ Drive/Colab\ Notebooks/data/
# %rm -rf gaf_output
# %mkdir gaf_output
# %cd gaf_output
# !unzip -q /content/gaf_output.zip

## Creating a GAF Cube

In [ ]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=[
    go.Mesh3d(
        # 8 vertices of a cube
        x=[0, 0, 1, 1, 0, 0, 1, 1],
        y=[0, 1, 1, 0, 0, 1, 1, 0],
        z=[0, 0, 0, 0, 1, 1, 1, 1],
        color = 'red',
        # vertexcolor = ['rgb(249, 65, 68)', 'rgb(243, 114, 44)','rgb(248, 150, 30)'],
        # facecolor = ['rgb(249, 65, 68)', 'rgb(243, 114, 44)','rgb(248, 150, 30)', 'rgb(249, 132, 74)',
        #              'rgb(249, 199, 79)', 'rgb(144, 190, 109)','rgb(67, 170, 139)', 'rgb(77, 144, 142)',
        #              'rgb(87, 117, 144)', 'rgb(39, 125, 161)','rgb(99, 65, 250)', 'rgb(153, 44, 242)'],
        colorbar_title='z',
        # colorscale=[[0, 'gold'],
        #             [0.5, 'mediumturquoise'],
        #             [0.5, 'magenta']],
        # Intensity of each vertex, which will be interpolated and color-coded
        # intensity = np.linspace(0, 1, 8, endpoint=True),
        # i, j and k give the vertices of triangles
        i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
        j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
        k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
        name='power',
        showscale=True,
        showlegend=True
    )
])

# fig.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from os import listdir
from os.path import isfile, join
files_path = '/content/gaf_output/training/normal/'
list_of_files = sorted([f for f in listdir(files_path) if isfile(join(files_path, f))])
list_of_files = list_of_files[0:6]
print(list_of_files)

imgs = []
for index, img in enumerate(list_of_files):
  print(index)
  _img = plt.imread(files_path + img)
  _img = _img[:,:, 2]
  imgs.append(_img)

L, C = imgs[0].shape

['2022_04_22 13:00:00.png', '2022_04_22 17:00:00.png', '2022_04_22 18:00:00.png', '2022_04_22 19:00:00.png', '2022_04_22 20:00:00.png', '2022_04_22 21:00:00.png']
0
1
2
3
4
5


In [ ]:
# pl_img=my_img[:,:, 2] #
# # assert L==C


In [ ]:
# plotly_blue='rgb(68, 122, 219)'# the  blue color in Plotly logo

In [ ]:
import plotly as py
from plotly.graph_objs import *


In [ ]:
pl_scl=[ [0.0, 'cyan'],
         [0.5, 'green' ],
         [0.75, 'yellow' ],
         [1.0, 'red' ]]


In [ ]:
x=np.linspace(0, L-1,  L)
y=np.linspace(0, L-1, L)
X, Y = np.meshgrid(x, y)


In [ ]:
zm=np.zeros(X.shape)
zM=(L-1)*np.ones(X.shape)


In [ ]:
def make_cube_face(x,y,z, colorscale=pl_scl, is_scl_reversed=False,
                   surfacecolor=imgs[0], text='Plotly cube'):
    return Surface(x=x, y=y, z=z,
                   colorscale='Rainbow',
                   showscale=True,
                   reversescale=True,
                   surfacecolor=surfacecolor,
                   text=text,
                   hoverinfo='text',
                  )


In [ ]:
trace_zm=make_cube_face(x=X, y=Y, z=zm, surfacecolor=imgs[0], text=list_of_files[0])
trace_zM=make_cube_face(x=X, y=Y, z=zM, surfacecolor=imgs[5], text=list_of_files[5])
trace_xm=make_cube_face(x=zm, y=Y, z=X, surfacecolor=imgs[2], text=list_of_files[2])
trace_xM=make_cube_face(x=zM, y=Y, z=X, surfacecolor=imgs[3], text=list_of_files[3])
trace_ym=make_cube_face(x=Y, y=zm, z=X, surfacecolor=imgs[4], text=list_of_files[4])
trace_yM=make_cube_face(x=Y, y=zM, z=X, surfacecolor=imgs[1], text=list_of_files[1])


In [ ]:
noaxis=dict(
            showbackground=False,
            showgrid=False,
            showline=False,
            showticklabels=False,
            ticks='',
            title='',
            zeroline=False)


In [ ]:
min_val=-0.01
max_val=L-1+0.01


In [ ]:
layout = Layout(
         title="",
         width=500,
         height=500,
         scene=Scene(xaxis=XAxis(noaxis, range=[min_val, max_val]),
                     yaxis=YAxis(noaxis, range=[min_val, max_val]),
                     zaxis=ZAxis(noaxis, range=[min_val, max_val]),
                     aspectratio=dict(x=1,
                                      y=1,
                                      z=1
                                     ),
                     camera=dict(eye=dict(x=-1.25, y=-1.25, z=1.25)),
                    ),

        paper_bgcolor='rgb(255,255,255)',
        hovermode='closest',
        margin=dict(t=50)
        )


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:550: DeprecationWarning:

plotly.graph_objs.XAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.XAxis
  - plotly.graph_objs.layout.scene.XAxis


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:578: DeprecationWarning:

plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:603: DeprecationWarning:

plotly.graph_objs.ZAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.scene.ZAxis


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:494: DeprecationWarning:

plotly.graph_objs.Scene is deprecated.
Please replace it with one of the following more spec

In [ ]:
fig=Figure(data=Data([trace_zm, trace_zM, trace_xm, trace_xM, trace_ym, trace_yM]), layout=layout)
fig.show()

In [ ]:
# Export to HTML for Home Assistant Integration
import plotly.express as px
fig.write_html("/content/gaf.html")